### IMPORT LIBRARY

In [1]:
from office365.runtime.auth.authentication_context import AuthenticationContext
from office365.sharepoint.client_context import ClientContext
from office365.sharepoint.files.file import File
from office365.runtime.auth.client_credential import ClientCredential
from office365.runtime.client_request_exception import ClientRequestException
import datetime
import pytz
import pandas as pd
import numpy as np
import csv
import glob
import pyodbc
import os 
import json
from io import BytesIO
import io
import platform
from function.PyToSp import *
from tqdm import tqdm
from urllib.parse import quote_plus
import numpy as np
import pandas as pd
from sqlalchemy import create_engine, event
import pyodbc
import requests
import inspect
from validate import *
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from urllib.parse import quote_plus
import msal
from itertools import chain
from send_email import *

### CONNECT TO AZURE SQL

In [2]:
import json
f = open ('database_information.json', "r")
qq = json.loads(f.read())
f.close()
ini_cnt_str ='Driver={driver_str};Server=tcp:hkazdevsqld3vnreserch.database.windows.net,1433;database={database};Uid={username};Pwd={password};Encrypt=yes;Authentication=ActiveDirectoryPassword;Connection Timeout=30;'.format(**qq)
quoted = quote_plus(ini_cnt_str)
cnt_str = 'mssql+pyodbc:///?odbc_connect={}'.format(quoted)
engine = create_engine(cnt_str)

### CONNECT TO SHAREPOINT

In [3]:
header_BIHub = 'share_point_BIHub'
config_BIHub = read_config_json(config_path, header_BIHub)
BIHub = SharePoint(config_BIHub)
BIHub.check_connect()

Web site title: S22M Research & BI Hub _ Successful Connection!


In [4]:
#Tất cả các tỉnh
relative_url = "/sites/BIHub/Shared Documents/Advisory Data/Dim_Dictionary"
sp_object = relative_url.split('/')[2].replace('-','')
list_folder = eval(sp_object).get_content_url(relative_url,return_list_folder=True)

Folder name: /sites/BIHub/Shared Documents/Advisory Data/Dim_Dictionary/Developer


In [5]:
#-------------------------------------------------------
df_summ_file = pd.DataFrame({'Name':[],'ServerRelativeUrl':[], 'TimeLastModified':[], 'ModTime':[], 'Modified_by_ID':[]})
for i in list_folder:
    df_summ_file = pd.concat([df_summ_file, eval(sp_object).get_content_url(i)])
list_file = df_summ_file['ServerRelativeUrl'].to_list()
#History file
df_query=pd.DataFrame(df_summ_file).reset_index(drop=True)
df_summ_file = df_summ_file.reset_index(drop=True)

Folder name: 
Files name: /sites/BIHub/Shared Documents/Advisory Data/Dim_Dictionary/Developer/FF_Developer_Dictionary.csv
Files name: /sites/BIHub/Shared Documents/Advisory Data/Dim_Dictionary/Developer/Developer_Dictionary.xlsx


In [6]:
url = []
for i in list_file:
    Folder = i.split('/')[7]
    if Folder == 'Developer_Dictionary.xlsx':
        url.append(i)
url        

['/sites/BIHub/Shared Documents/Advisory Data/Dim_Dictionary/Developer/Developer_Dictionary.xlsx']

In [7]:
def get_data(relative_url, file_url):
    header_BIHub = 'share_point_BIHub'
    config_BIHub = read_config_json(config_path, header_BIHub)
    BIHub = SharePoint(config_BIHub)

    sp_object = relative_url.split('/')[2].replace('-', '')
    data_MS = BIHub.get_file_with_sheet_name(file_url, sheet_name='MainSector_Developer')
    data_IP = BIHub.get_file_with_sheet_name(file_url, sheet_name='IP_Developer')
    data = pd.concat([data_MS, data_IP], ignore_index=True)
    # Convert data to DataFrame if it's not already
    if not isinstance(data, pd.DataFrame):
        data = pd.DataFrame(data)

    # Remove irrelevant data
    data.dropna(axis=0, how='all', inplace=True)  # remove blank rows
    data.drop(data.columns[data.columns.str.contains('Unnamed')], axis=1, inplace=True)  # remove blank columns
    data.columns = data.columns.str.strip().str.replace(r'\s+', '', regex=True)  # strip column
    return data


## Import

In [374]:
#Create empty df for checking dictionary
df_dict = pd.DataFrame(columns=['File_Name', 'Missing_Values', 'Flag'])
df_temp_flat = pd.DataFrame()
df_flat = pd.DataFrame()
#-------------------------------------------------------
'''Get data''' 
table_name = 'Developer_Dictionary'
columns_that_need_unidecode = ['Cleaned_Developer']

for file_url in tqdm(url):
    data = get_data(relative_url, file_url)
    data = remove_unformated_character(data)
    data = pd.DataFrame(data)
    for i in columns_that_need_unidecode:
        data[i] = remove_unicode(data[i])
    today = pd.to_datetime('today').strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
   
    hold_data = data[data['Update_Developer'].isnull()]
    new_row = data[data['Update_Developer'].notnull()]
    new_row['Cleaned_Developer'] = new_row['Update_Developer']
    new_row['Last_Update'] = today
    new_row = new_row.drop('Update_Developer', axis=1)
    hold_data = hold_data.drop('Update_Developer', axis=1)
    data = pd.concat([hold_data, new_row], ignore_index=True)
    # filtered_data = data[~data['Raw_Developer'].isin(pd.read_sql_query("SELECT Raw_Developer FROM GENERAL.Developer_Dictionary", engine)['Raw_Developer'])]
    # get_db= pd.read_sql_query("SELECT * FROM GENERAL.Developer_Dictionary", engine)
    # df_get_db = pd.DataFrame(get_db)
    # filtered_data = data[~data['Raw_Developer'].isin(df_get_db['Raw_Developer'])]
    data = data.dropna(subset=['Raw_Developer'])
    data = data.drop('Update_Developer', axis=1)
    data.to_sql(table_name, engine, index=False, if_exists='replace', schema='GENERAL')

  0%|          | 0/1 [00:00<?, ?it/s]

### Update

In [14]:
columns_that_need_unidecode = ['Cleaned_Developer']

# Define SQL Server connection details
engine = create_engine(cnt_str, pool_size=10, max_overflow=20)

# Iterate over URLs
for file_url in tqdm(url):
    data = get_data(relative_url, file_url)
    data = remove_unformated_character(data)
    data = pd.DataFrame(data)
    
    # Iterate over columns that need unidecode
    for i in columns_that_need_unidecode:
        data[i] = remove_unicode(data[i])

    today = pd.to_datetime('today').strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
    new_row = data[data['Update_Developer'].notnull()]
    new_row['Cleaned_Developer'] = new_row['Update_Developer']
    new_row['Last_Update'] = today

    # Iterate over rows in the new_row DataFrame and update the database
    for index, row in new_row.iterrows():
        sql_query = """
            UPDATE GENERAL.Developer_Dictionary
            SET Cleaned_Developer = :cleaned_dev,
                Last_Update = :last_update
            WHERE Raw_Developer = :raw_dev AND Sector = :sector
        """

        # Execute the query with the row values using parameterized query
        with engine.connect() as conn:
            conn.execute(sql_query, cleaned_dev=row['Cleaned_Developer'],
                                   last_update=today,
                                   raw_dev=row['Raw_Developer'],
                                   sector=row['Sector'])

  0%|          | 0/1 [00:00<?, ?it/s]C:\Users\pnguyenhaiduong\AppData\Local\Temp\ipykernel_25088\3842369159.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_row['Cleaned_Developer'] = new_row['Update_Developer']
C:\Users\pnguyenhaiduong\AppData\Local\Temp\ipykernel_25088\3842369159.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_row['Last_Update'] = today
  0%|          | 0/1 [00:05<?, ?it/s]


ProgrammingError: (pyodbc.ProgrammingError) ('The SQL contains 0 parameter markers, but 1 parameters were supplied', 'HY000')
[SQL: 
            UPDATE GENERAL.Developer_Dictionary
            SET Cleaned_Developer = :cleaned_dev,
                Last_Update = :last_update
            WHERE Raw_Developer = :raw_dev AND Sector = :sector
        ]
[parameters: {'cleaned_dev': '379 Urban Development', 'last_update': '2023-12-06 16:43:44.934', 'raw_dev': '379 Urban Development Ltd.', 'sector': 'Main Sector'}]
(Background on this error at: https://sqlalche.me/e/14/f405)